# Objectives:

1) This script should convert all protein sequences found within .gb files in a given parent directory, walk through the parent directory, and output all orthologous protein sequences into a formatted .faa file for downstream MSA.

2) This script will also output both individual .faa files for orthologs with more than one isoform (e.g. protein A has alpha/beta forms and you will get both protein_A_alpha.faa & protein_A_beta.faa files) as well as a combined file containing all isoforms (e.g. protein_A_combined.faa will have alpha, beta, delta, etc isoforms together).

### Notes before running:
1) You must provide a parent directory containing .gb files to walk through 
2) You must specify an output directory for the .faa files

In [1]:
#Librarying/installing
#!pip install glob
#!pip install Bio
import os
import glob
import re
from Bio import SeqIO

In [ ]:

def natural_key(text):
    """
    Generate a key for natural sorting.
    Splits the text into strings and integers so that numeric parts are sorted in numerical order.
    """
    return [int(c) if c.isdigit() else c.lower() for c in re.split('(\d+)', text)]

def extract_orthologs(parent_dir, output_dir):
    """
    Recursively searches for .gb files in parent_dir, extracts CDS features with translations,
    groups them by both isoform (full identifier) and base protein (identifier before the last underscore),
    and writes out FASTA files.
    
    For each isoform group, a file named <isoform_key>.faa is created.
    For base protein groups that have multiple isoforms, a combined file named <base_key>_combined.faa is created.
    In the combined file, each FASTA header includes the full isoform identity.
    Sequences are sorted in ascending order by species name (using natural sort).
    """
    # Dictionary to store sequences by full isoform key.
    isoform_dict = {}
    # Dictionary to store sequences by base protein key.
    # Now storing tuples of (species, sequence, isoform_key) so that we can later retain isoform identity.
    base_dict = {}
    # Dictionary mapping a base protein to the set of isoform keys encountered.
    base_to_isoforms = {}

    # Recursively find all .gb files.
    gb_files = glob.glob(os.path.join(parent_dir, '**', '*.gb'), recursive=True)
    print(f"Found {len(gb_files)} GenBank files in {parent_dir}")

    for gb_file in gb_files:
        try:
            for record in SeqIO.parse(gb_file, "genbank"):
                species = record.annotations.get("organism", "Unknown_species")
                for feature in record.features:
                    if feature.type == "CDS" and "translation" in feature.qualifiers:
                        # Choose an identifier: prefer 'gene', else use 'product'
                        if "gene" in feature.qualifiers:
                            ortholog_key = feature.qualifiers["gene"][0]
                        elif "product" in feature.qualifiers:
                            ortholog_key = feature.qualifiers["product"][0]
                        else:
                            continue  # Skip if no identifier available.

                        # Clean the key (e.g., remove spaces).
                        ortholog_key_clean = ortholog_key.replace(" ", "_")
                        
                        # Store in the isoform dictionary.
                        isoform_dict.setdefault(ortholog_key_clean, []).append(
                            (species, feature.qualifiers["translation"][0])
                        )
                        
                        # Determine the base key: if an underscore is present, take everything before the last underscore.
                        if "_" in ortholog_key_clean:
                            parts = ortholog_key_clean.rsplit("_", 1)
                            base_key = parts[0]
                        else:
                            base_key = ortholog_key_clean
                        
                        # Store in the base dictionary, now also storing the isoform identity.
                        base_dict.setdefault(base_key, []).append(
                            (species, feature.qualifiers["translation"][0], ortholog_key_clean)
                        )
                        # Track which isoforms belong to each base key.
                        base_to_isoforms.setdefault(base_key, set()).add(ortholog_key_clean)
        except Exception as e:
            print(f"Error processing {gb_file}: {e}")

    # Ensure the output directory exists.
    os.makedirs(output_dir, exist_ok=True)
    
    # Write FASTA files for each isoform group.
    for isoform_key, entries in isoform_dict.items():
        # Sort entries by species using natural sort.
        sorted_entries = sorted(entries, key=lambda x: natural_key(x[0]))
        output_file = os.path.join(output_dir, f"{isoform_key}.faa")
        with open(output_file, "w") as f:
            for species, sequence in sorted_entries:
                f.write(f">{species}_{isoform_key}\n")
                f.write(f"{sequence}\n")
        print(f"Wrote {len(entries)} sequences to {output_file}")
    
    # Write combined FASTA file only for base proteins with multiple isoforms.
    for base_key, entries in base_dict.items():
        # Check the number of distinct isoforms for this base protein.
        if len(base_to_isoforms.get(base_key, [])) > 1:
            # Sort entries by species using natural sort.
            sorted_entries = sorted(entries, key=lambda x: natural_key(x[0]))
            output_file = os.path.join(output_dir, f"{base_key}_combined.faa")
            with open(output_file, "w") as f:
                for species, sequence, isoform in sorted_entries:
                    # Include the isoform identity in the header.
                    f.write(f">{species}_{isoform}\n")
                    f.write(f"{sequence}\n")
            print(f"Wrote {len(entries)} sequences to {output_file} (combined isoforms)")
        else:
            print(f"Skipping combined file for {base_key} as it has only one isoform.")

#Time to use the function :)
parent_dir = "/Path/To/Your/Parent/Directory"
output_dir = "/Path/To/Your/Output/Directory"
extract_orthologs(parent_dir, output_dir)